<a href="https://colab.research.google.com/github/wonjae124/2021_Lab_project/blob/main/Transfer_learning_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
from torchsummary import summary

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


# 사전 학습 모델 불러와서 GPU에 올리기

In [16]:
import torchvision.models as models
alexnet = models.alexnet().to(device)
resnet18 = models.resnet18().to(device)
vgg16 = models.vgg16().to(device)
densenet = models.densenet161().to(device) 
inception = models.inception_v3().to(device)
googlenet = models.googlenet().to(device)
shufflenet = models.shufflenet_v2_x1_0().to(device)
mobilenet_v2 = models.mobilenet_v2().to(device)
wide_resnet50_2 = models.wide_resnet50_2().to(device)
mnasnet = models.mnasnet1_0().to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:80: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


In [4]:
# CIFAR10을 위한 ResNet18을 불러오기

In [17]:
model1 = torchvision.models.resnet18(pretrained=True).to(device)
summary(model1,(3,7,7))

#ResNet, VGG16, VGG19는 224 by 224 사이즈의 이미지이므로 커널 사이즈가 7 by 7이고 맥스풀링은 사용한다.
#보통, CIFAR10은 이미지가 28 by 28 이므로 크기가 작아서 커널 사이즈를 3 by 3, 맥스풀링 없이 모델 구조 만든다.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 4, 4]           9,408
       BatchNorm2d-2             [-1, 64, 4, 4]             128
              ReLU-3             [-1, 64, 4, 4]               0
         MaxPool2d-4             [-1, 64, 2, 2]               0
            Conv2d-5             [-1, 64, 2, 2]          36,864
       BatchNorm2d-6             [-1, 64, 2, 2]             128
              ReLU-7             [-1, 64, 2, 2]               0
            Conv2d-8             [-1, 64, 2, 2]          36,864
       BatchNorm2d-9             [-1, 64, 2, 2]             128
             ReLU-10             [-1, 64, 2, 2]               0
       BasicBlock-11             [-1, 64, 2, 2]               0
           Conv2d-12             [-1, 64, 2, 2]          36,864
      BatchNorm2d-13             [-1, 64, 2, 2]             128
             ReLU-14             [-1, 6

In [18]:
for j, (name,param) in enumerate(model1.named_parameters()): # 레이어의 이름 확인하기. 
    print(j, name)

0 conv1.weight
1 bn1.weight
2 bn1.bias
3 layer1.0.conv1.weight
4 layer1.0.bn1.weight
5 layer1.0.bn1.bias
6 layer1.0.conv2.weight
7 layer1.0.bn2.weight
8 layer1.0.bn2.bias
9 layer1.1.conv1.weight
10 layer1.1.bn1.weight
11 layer1.1.bn1.bias
12 layer1.1.conv2.weight
13 layer1.1.bn2.weight
14 layer1.1.bn2.bias
15 layer2.0.conv1.weight
16 layer2.0.bn1.weight
17 layer2.0.bn1.bias
18 layer2.0.conv2.weight
19 layer2.0.bn2.weight
20 layer2.0.bn2.bias
21 layer2.0.downsample.0.weight
22 layer2.0.downsample.1.weight
23 layer2.0.downsample.1.bias
24 layer2.1.conv1.weight
25 layer2.1.bn1.weight
26 layer2.1.bn1.bias
27 layer2.1.conv2.weight
28 layer2.1.bn2.weight
29 layer2.1.bn2.bias
30 layer3.0.conv1.weight
31 layer3.0.bn1.weight
32 layer3.0.bn1.bias
33 layer3.0.conv2.weight
34 layer3.0.bn2.weight
35 layer3.0.bn2.bias
36 layer3.0.downsample.0.weight
37 layer3.0.downsample.1.weight
38 layer3.0.downsample.1.bias
39 layer3.1.conv1.weight
40 layer3.1.bn1.weight
41 layer3.1.bn1.bias
42 layer3.1.conv2.wei

In [19]:
model1.fc.in_features

512

In [20]:
model2 = torchvision.models.resnet18(pretrained=True)

model2.conv1 = nn.Conv2d(3,64, kernel_size = 3, stride = 1, padding = 1)

num_ftrs = model2.fc.in_features # 512 

model2.fc = nn.Linear(num_ftrs,10) # 512 -> 10으로 바꾸자

model2 = model2.to(device)

In [21]:
model2.fc.in_features

512

In [22]:
summary(model2,(3,7,7))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 7, 7]           1,792
       BatchNorm2d-2             [-1, 64, 7, 7]             128
              ReLU-3             [-1, 64, 7, 7]               0
         MaxPool2d-4             [-1, 64, 4, 4]               0
            Conv2d-5             [-1, 64, 4, 4]          36,864
       BatchNorm2d-6             [-1, 64, 4, 4]             128
              ReLU-7             [-1, 64, 4, 4]               0
            Conv2d-8             [-1, 64, 4, 4]          36,864
       BatchNorm2d-9             [-1, 64, 4, 4]             128
             ReLU-10             [-1, 64, 4, 4]               0
       BasicBlock-11             [-1, 64, 4, 4]               0
           Conv2d-12             [-1, 64, 4, 4]          36,864
      BatchNorm2d-13             [-1, 64, 4, 4]             128
             ReLU-14             [-1, 6

# Model Freezing

In [33]:
model4 = torchvision.models.alexnet(pretrained=True)
print(f"origin {model4.classifier[6]}")

num_ftrs = model4.classifier[6].in_features
model4.classifier[6] = nn.Linear(num_ftrs, 10) #피처 추출 부분은 프리징 시키고 분류 파트(출력 레이어의 노드)를 클래수 수와 맞춰 준다.

print(f"After {model4.classifier[6]}")

model4 = model4.to(device)

origin Linear(in_features=4096, out_features=1000, bias=True)
After Linear(in_features=4096, out_features=10, bias=True)


In [34]:
for i,(name,param) in enumerate(model4.named_parameters()): #모델 파라미터명 확인
    print(i, name)
#피처 추출 부분은 프리징 시키고 분류 파트(출력 레이어의 노드)를 클래수 수와 맞춰 준다.
#0~9번은 feature 추출 부분
# 10 ~ 15 부분은 분류 부분

0 features.0.weight
1 features.0.bias
2 features.3.weight
3 features.3.bias
4 features.6.weight
5 features.6.bias
6 features.8.weight
7 features.8.bias
8 features.10.weight
9 features.10.bias
10 classifier.1.weight
11 classifier.1.bias
12 classifier.4.weight
13 classifier.4.bias
14 classifier.6.weight
15 classifier.6.bias


# 피쳐 부분 프리징

In [37]:
for i,(name,param) in enumerate(model4.named_parameters()):
    print(i, name, param.requires_grad) 

0 features.0.weight True
1 features.0.bias True
2 features.3.weight True
3 features.3.bias True
4 features.6.weight True
5 features.6.bias True
6 features.8.weight True
7 features.8.bias True
8 features.10.weight True
9 features.10.bias True
10 classifier.1.weight True
11 classifier.1.bias True
12 classifier.4.weight True
13 classifier.4.bias True
14 classifier.6.weight True
15 classifier.6.bias True


In [13]:
for i,(name,param) in enumerate(model4.named_parameters()):
    param.requires_grad = False 
    if i == 9:      #0~9까지 False로 프리징 시켰다.
        print('end')
        break

end


# Requires_grad = False 확인하기

In [14]:
f_list = [0, 3, 6, 8, 10] 
c_list = [1, 4, 6]
for i in f_list:
    print(model4.features[i].weight.requires_grad)
    print(model4.features[i].bias.requires_grad)
for j in c_list:
    print(model4.classifier[j].weight.requires_grad)
    print(model4.classifier[j].bias.requires_grad)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
